In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import shutil
import glob
from sklearn.model_selection import train_test_split
import cv2, hashlib
from tqdm import tqdm


INP_DIR = "data/validation"

TRAIN_DIR = "data/validation_processed/training"
TEST_DIR  = "data/validation_processed/testing"
PROC_DIR  = "data/validation_processed/unsorted"

SEED = 136
TRAIN_TEST_SPLIT = 0.2
IMG_SIZE = (100, 100)
BATCH_SIZE = 32

In [ ]:
target_dirs = [TRAIN_DIR, TEST_DIR, PROC_DIR]
sub_dirs = ["0_invalid", "1_tec_results"]

print("Creating directory structure...")
for main_dir in target_dirs:
    for sub in sub_dirs:
        os.makedirs(os.path.join(main_dir, sub), exist_ok=True)
print("Done.")

Creating directory structure...
Done.


In [ ]:
def bw_image(path_in:str, class_dir:str, name:str) -> None:
    try:
        img = Image.open(path_in)
        bw_img = img.convert("L") 
        bw_img.save(f"{PROC_DIR}/{class_dir}/bw_{name}")
    except Exception as e:
        print(f"Error processing {name}: {e}")

In [112]:
print("Processing images to grayscale...")
count = 0

for class_dir in os.listdir(INP_DIR):
    path = f"{INP_DIR}/{class_dir}"
    if not os.path.isdir(path): continue 
    
    for file in os.listdir(path):
        bw_image(f"{path}/{file}", class_dir, file)
        count += 1

print(f"Processed {count} images.")

Processing images to grayscale...
Processed 185 images.


In [ ]:
print("Indexing and splitting data...")
files = []
labels = []

for class_dir in sub_dirs:
    class_path = os.path.join(PROC_DIR, class_dir)
    img_files = glob.glob(f"{class_path}/*")
    
    label = 1 if "1_tec_results" in class_dir else 0
    
    files.extend(img_files)
    labels.extend([label] * len(img_files))

train_val_files, test_files, _, _ = train_test_split(
    files, labels, test_size=TRAIN_TEST_SPLIT, random_state=SEED, stratify=labels
)

print(f"Copying {len(train_val_files)} training files...")
for file_path in train_val_files:
    new_path = file_path.replace("unsorted", "training") 
    shutil.copy(file_path, new_path)

print(f"Copying {len(test_files)} testing files...")
for file_path in test_files:
    new_path = file_path.replace("unsorted", "testing")
    shutil.copy(file_path, new_path)

print("Data split complete.")

Indexing and splitting data...
Copying 148 training files...
Copying 37 testing files...
Data split complete.


In [ ]:
print("Loading datasets from disk...")

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale', # Forces (100, 100, 1) shape
    label_mode="binary"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    label_mode="binary"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    label_mode="binary"
)

print(f"Classes found: {train_ds.class_names}")

Loading datasets from disk...
Found 148 files belonging to 2 classes.
Using 119 files for training.
Found 148 files belonging to 2 classes.
Using 29 files for validation.
Found 37 files belonging to 2 classes.
Classes found: ['0_invalid', '1_tec_results']


In [ ]:
def build_micro_cnn(input_shape=(100, 100, 1)):
    model = models.Sequential([
        layers.Rescaling(1./255, input_shape=input_shape),

        layers.RandomContrast(0.1),
        layers.RandomTranslation(height_factor=0.05, width_factor=0.05),

        # Convolutional Blocks
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Classification Head
        layers.Flatten(),
        layers.Dense(64, activation='relu'),

        layers.Dropout(0.5),

        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_micro_cnn()
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_8 (Rescaling)     (None, 100, 100, 1)       0         
                                                                 
 random_contrast_3 (RandomC  (None, 100, 100, 1)       0         
 ontrast)                                                        
                                                                 
 random_translation_3 (Rand  (None, 100, 100, 1)       0         
 omTranslation)                                                  
                                                                 
 conv2d_24 (Conv2D)          (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 49, 49, 32)        0         
 ng2D)                                                           
                                                      

In [120]:
print("Starting training...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    class_weight={0: 1.0, 1: 5.0}
)
print("Training finished!")

Starting training...
Epoch 1/50
4/4 [==============================] - 1s 47ms/step - loss: 0.2954 - accuracy: 0.9916 - val_loss: 0.4960 - val_accuracy: 0.8621
Epoch 2/50
4/4 [==============================] - 0s 46ms/step - loss: 0.4388 - accuracy: 0.9580 - val_loss: 0.6954 - val_accuracy: 0.8621
Epoch 3/50
4/4 [==============================] - 0s 50ms/step - loss: 0.3889 - accuracy: 0.9748 - val_loss: 0.6547 - val_accuracy: 0.8276
Epoch 4/50
4/4 [==============================] - 0s 46ms/step - loss: 0.3718 - accuracy: 0.9160 - val_loss: 0.4087 - val_accuracy: 0.8276
Epoch 5/50
4/4 [==============================] - 0s 49ms/step - loss: 0.5386 - accuracy: 0.9244 - val_loss: 0.3776 - val_accuracy: 0.8966
Epoch 6/50
4/4 [==============================] - 0s 54ms/step - loss: 0.3748 - accuracy: 0.9748 - val_loss: 0.3349 - val_accuracy: 0.8621
Epoch 7/50
4/4 [==============================] - 0s 50ms/step - loss: 0.4665 - accuracy: 1.0000 - val_loss: 0.3907 - val_accuracy: 0.7931
Epoch 

In [168]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc*100:.2f}%")

2/2 [==============================] - 0s 10ms/step - loss: 0.2040 - accuracy: 0.9189
Test Accuracy: 91.89%


In [169]:
def reset_folders():
    folders_to_clean = [TRAIN_DIR, TEST_DIR, PROC_DIR]
    for folder in folders_to_clean:
        if os.path.exists(folder):
            shutil.rmtree(folder)
            print(f"Cleaned {folder}")
    
    for main_dir in folders_to_clean:
        for sub in ["0_invalid", "1_tec_results"]:
            os.makedirs(os.path.join(main_dir, sub), exist_ok=True)
    print("Folders are ready for a fresh run!")

reset_folders()

Cleaned data/validation_processed/training
Cleaned data/validation_processed/testing
Cleaned data/validation_processed/unsorted
Folders are ready for a fresh run!


### Video Watching Script

In [166]:
INPUT_FOLDER:str = os.path.expanduser("data/videos")
OUTPUT_FOLDER:str = "data/found_matches"
CONFIDENCE_THRESHOLD:float = 0.92
SKIP_SECONDS:float = 2
COOLDOWN_SECONDS:int = 90

In [ ]:
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

videos = []
videos.extend(glob.glob(os.path.join(INPUT_FOLDER, "*.mp4")))

def preprocess_frame(frame):
    """Matches the training preprocessing: 100x100, Grayscale (0-255)"""
    # Resize to img input size
    small = cv2.resize(frame, (100, 100))
    gray = cv2.cvtColor(small, cv2.COLOR_BGR2GRAY)
    
    # Add Batch & Channel Dimensions
    input_tensor = np.expand_dims(gray, axis=0)
    input_tensor = np.expand_dims(input_tensor, axis=-1)

    return input_tensor

for VIDEO_PATH in videos:
    print(f"Scanning video: {VIDEO_PATH}")
    cap = cv2.VideoCapture(VIDEO_PATH)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    COOLDOWN_FRAMES:int = int(COOLDOWN_SECONDS*fps)
    SKIP_FRAMES:int = int(SKIP_SECONDS*fps)

    current_frame = 0
    rounds_found = 0
    cooldown_timer = 0

    print(f"Video FPS: {fps} | Total Frames: {total_frames}")

    pbar = tqdm(total=total_frames, desc=f"Scanning {VIDEO_PATH}", unit="frames")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # End of video
        
        if cooldown_timer > 0:
            cooldown_timer -= 1
        elif current_frame % SKIP_FRAMES == 0:
            
            input_tensor = preprocess_frame(frame)
            
            prediction = model.predict(input_tensor, verbose=0)[0][0]
            
            if prediction > CONFIDENCE_THRESHOLD:
                # Results screen found
                timestamp_sec = int(current_frame / fps)
                print(f"[{timestamp_sec}s] Round Found! (Conf: {prediction:.2f})")

                raw_id = f"{os.path.basename(VIDEO_PATH)}_{timestamp_sec}"
                unique_hash = hashlib.sha256(raw_id.encode()).hexdigest()[:8]
                
                filename = f"match_{rounds_found:03d}_{timestamp_sec}s_{unique_hash}.jpg"
                cv2.imwrite(os.path.join(OUTPUT_FOLDER, filename), frame)
                
                rounds_found += 1
                pbar.set_postfix(found=rounds_found)
                
                cooldown_timer = COOLDOWN_FRAMES

        current_frame += 1
        pbar.update(1)

    pbar.close()
    cap.release()
    print(f"Done! Found {rounds_found} rounds.")

Scanning video: data/videos/Battling Zone in Zone Battle ： Zone Battle - Tetris® Effect： Connected.mp4
Video FPS: 60.0 | Total Frames: 1142200


Scanning data/videos/Battling Zone in Zone Battle ： Zone Battle - Tetris® Effect： Connected.mp4:   1%|          | 13191/1142200 [00:24<32:11, 584.62frames/s, found=1]

[218s] Match Found! (Conf: 0.93)


Scanning data/videos/Battling Zone in Zone Battle ： Zone Battle - Tetris® Effect： Connected.mp4:   2%|▏         | 23971/1142200 [00:47<43:57, 423.93frames/s, found=2]  

[398s] Match Found! (Conf: 1.00)


...

[1590s] Match Found! (Conf: 0.99)


Scanning data/videos/Zone Battle only part 24 (Tetris Effect) {my thoughts about Marvel Tokon}.mp4:  88%|████████▊ | 104753/118953 [07:43<01:16, 186.38frames/s, found=10]

[1747s] Match Found! (Conf: 0.99)


Scanning data/videos/Zone Battle only part 24 (Tetris Effect) {my thoughts about Marvel Tokon}.mp4: 100%|██████████| 118953/118953 [08:46<00:00, 226.07frames/s, found=10]

Done! Found 10 matches.


In [162]:
pbar.close()
cap.release()